In [9]:
!pip install scikit-surprise
# %pip install kagglehub

Denne fil bliver brugt til at analysere og kigge på det datasæt vi har valgt at lave vores endelige projekt i. 



In [10]:
import kagglehub
import os
import pandas as pd

# Download latest version
path = kagglehub.dataset_download("dsabljic/movielens-32m")
print("Path to dataset files:", path)

# List subfolder 'ml-32m'
dataset_folder = os.path.join(path, "ml-32m")
print("Indhold i mappen ml-32m:")
print(os.listdir(dataset_folder))

# Find en csv-fil i mappen
csv_files = [f for f in os.listdir(dataset_folder) if f.endswith('.csv')]



# Læs første CSV-fil (bare som eksempel)
if csv_files:
    links_csv = os.path.join(dataset_folder, csv_files[0])
    df_links = pd.read_csv(links_csv)

    movies_csv = os.path.join(dataset_folder, "movies.csv")
    df_movies = pd.read_csv(movies_csv)

    ratings_csv = os.path.join(dataset_folder, "ratings.csv")
    df_ratings = pd.read_csv(ratings_csv)

    tags_csv = os.path.join(dataset_folder, "tags.csv")
    df_tags = pd.read_csv(tags_csv)
    
else:
    print("Ingen CSV-filer fundet i mappen.")

# kombiner datafiler til en enkelt DataFrame
df = pd.merge(df_ratings, df_movies, on='movieId', how='left')
# df = pd.merge(df, df_links, on='movieId', how='left')
print("DataFrame med ratings og movie info:")
print(df.head())
print("Antal rækker i DataFrame:", len(df))


Path to dataset files: C:\Users\eriks\.cache\kagglehub\datasets\dsabljic\movielens-32m\versions\1
Indhold i mappen ml-32m:
['checksums.txt', 'links.csv', 'movies.csv', 'ratings.csv', 'README.txt', 'tags.csv']
DataFrame med ratings og movie info:
   userId  movieId  rating  timestamp  \
0       1       17     4.0  944249077   
1       1       25     1.0  944250228   
2       1       29     2.0  943230976   
3       1       30     5.0  944249077   
4       1       32     5.0  943228858   

                                               title  \
0                       Sense and Sensibility (1995)   
1                           Leaving Las Vegas (1995)   
2  City of Lost Children, The (Cité des enfants p...   
3  Shanghai Triad (Yao a yao yao dao waipo qiao) ...   
4          Twelve Monkeys (a.k.a. 12 Monkeys) (1995)   

                                   genres  
0                           Drama|Romance  
1                           Drama|Romance  
2  Adventure|Drama|Fantasy|Mystery|Sci

In [12]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split, cross_validate
from surprise import accuracy
import pandas as pd


# Prepare data for Surprise (needs userId, itemId, rating columns)
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(df_ratings[['userId', 'movieId', 'rating']], reader)

# Split data into train and test set
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Define model
model = SVD()

# Train model
model.fit(trainset)

# Predict ratings for testset
predictions = model.test(testset)

# Evaluate performance
print("RMSE:", accuracy.rmse(predictions))
print("MAE:", accuracy.mae(predictions))

# Show some example predictions
print("\nExample predictions:")
for pred in predictions[:10]:
    print(f"User {pred.uid} → Movie {pred.iid} | True: {pred.r_ui:.1f} | Predicted: {pred.est:.2f}")


RMSE: 0.7717
RMSE: 0.7716991458079654
MAE:  0.5799
MAE: 0.5798519649942866

Example predictions:
User 157024 → Movie 2949 | True: 3.5 | Predicted: 3.57
User 120367 → Movie 6378 | True: 4.5 | Predicted: 4.37
User 43797 → Movie 176423 | True: 5.0 | Predicted: 3.64
User 29008 → Movie 3202 | True: 4.0 | Predicted: 2.80
User 11864 → Movie 3267 | True: 4.0 | Predicted: 3.70
User 55714 → Movie 8961 | True: 4.0 | Predicted: 4.10
User 126000 → Movie 50 | True: 3.5 | Predicted: 3.99
User 183292 → Movie 2770 | True: 2.5 | Predicted: 3.38
User 159884 → Movie 1 | True: 4.5 | Predicted: 4.31
User 62021 → Movie 1250 | True: 5.0 | Predicted: 4.45
